# COLLAB

In [ ]:
### loading the data
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import files

files.upload() #this will prompt you to upload the kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dbvieira","key":"ba29a2c62ed8d384d9df88b152219dde"}'}

In [ ]:
#Script to download and unpach the kaggle content
!ls -lha kaggle.json


!pip install -q kaggle




!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


!chmod 600 /root/.kaggle/kaggle.json


!pwd


!kaggle datasets list


!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria


!unzip cell-images-for-detecting-malaria.zip

In [ ]:
#Deleting useless content from storage
!rm -rf cell_images/cell_images
!rm cell_images/Parasitized/Thumbs.db
!rm cell_images/Uninfected/Thumbs.db

In [ ]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 6.8 MB/s eta 0:00:00


# HEADER

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras import mixed_precision
import matplotlib.pyplot as plt
import keras_tuner as kt
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import pathlib
from sklearn.metrics import confusion_matrix

In [2]:
# Suppress TPU messages which start with "Executing op"

import sys, re, logging

class Filter(object):
    def __init__(self, stream):
        self.stream = stream

    def __getattr__(self, attr_name):
        return getattr(self.stream, attr_name)

    def write(self, data):
        if not data.startswith("Executing op"):
            self.stream.write(data)
            self.stream.flush()

    def flush(self):
        self.stream.flush()
  
sys.stdout = Filter(sys.stdout)
sys.stderr = Filter(sys.stderr)

logger = logging.getLogger(__name__)

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

# If the following line doesn't output "Tesla T4", you can try getting access to
# another GPU by going to Runtime -> Factory Reset Runtime -> "Yes" and then
# re-running this cell.
!nvidia-smi -L

Num GPUs Available:  1
GPU 0: NVIDIA GeForce RTX 3050 Laptop GPU (UUID: GPU-2f6ca3a6-0a55-0ce5-cd9b-fde2db78c979)


In [4]:
data_dir = 'cell_images'


#Reading and spliting the dataset in training and validation using 80/20 cross-validation
image_size = (50,50)
batch_size = 32
validation_split = 0.2
color_mode = 'rgb'
seed = 123

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=validation_split,
  subset="training",
  seed=seed,
  shuffle = True,
  label_mode = 'categorical',
  image_size=image_size,
  color_mode = color_mode,
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=validation_split,
  subset="validation",
  seed= seed,
  label_mode = 'categorical',
  color_mode = color_mode,
  image_size=image_size,
  batch_size=batch_size)

#Printing Classes Names
class_names = train_ds.class_names
print('\nLabels:', class_names)

Found 27558 files belonging to 2 classes.
Using 22047 files for training.
Found 27558 files belonging to 2 classes.
Using 5511 files for validation.

Labels: ['Parasitized', 'Uninfected']


In [5]:
#Prefetching the dataset to improve the performance during the train
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Turn on mixed precision training to improve the processing time of the model
mixed_precision.set_global_policy("mixed_float16") # set global data policy to mixed precision 

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6


In [6]:
def preprocess_img(image, label):
  # image = image/255. # scale image values
  return tf.cast(image, tf.float32)/255, label

#preprocessing the datasets, normalizing the values
train_data = train_ds.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
val_data = val_ds.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

In [7]:
#creating model validation function to evaluate and compare models
def model_validation(model, val_data):
  # Check the accuracy of our model
  loss, accuracy = model.evaluate(val_data)
  print(f"Model loss on test set: {loss}")
  print(f"Model accuracy on test set: {(accuracy*100):.2f}%")

  # Make predictions
  y_preds = model.predict(val_data)

  id = 0
  temp = 0
  tempx = 0
  for x,y in val_data:
    if id == 0:
      tempx = x
      temp = y
      id+=1
    else:
      tempx = np.concatenate([tempx,x])
      temp = np.concatenate([temp, y])

  y_pred = (y_preds > 0.5)
  temp = (temp > 0.5)

  # Note: The following confusion matrix code is a remix of Scikit-Learn's 
  # plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
  # and Made with ML's introductory notebook - https://github.com/GokuMohandas/MadeWithML/blob/main/notebooks/08_Neural_Networks.ipynb 
  import itertools

  figsize = (10, 10)

  # Create the confusion matrix
  cm = confusion_matrix(temp[:,0],y_pred[:,0])
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0]

  # Let's prettify it
  fig, ax = plt.subplots(figsize=figsize)
  # Create a matrix plot
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # https://matplotlib.org/3.2.0/api/_as_gen/matplotlib.axes.Axes.matshow.html
  fig.colorbar(cax)

  # Create classes
  classes = True

  if classes:
    labels = class_names
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
        xlabel="Predicted label",
        ylabel="True label",
        xticks=np.arange(n_classes),
        yticks=np.arange(n_classes),
        xticklabels=labels,
        yticklabels=labels)

  # Set x-axis labels to bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Adjust label size
  ax.xaxis.label.set_size(20)
  ax.yaxis.label.set_size(20)
  ax.title.set_size(20)

  # Set threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
            horizontalalignment="center",
            color="white" if cm[i, j] > threshold else "black",
            size=15)

# MODEL CREATION

In [8]:
#Creating default model
def create_model():
  model = Sequential([
      layers.Conv2D(32, 3 ,padding= "valid", activation= "relu", input_shape = (50,50,3)),
        layers.MaxPooling2D(),
        layers.Dropout(0.2),

        layers.Conv2D(64, 3, padding= "valid", activation= "relu"),
        layers.MaxPooling2D(),
        layers.Dropout(0.2),

        layers.Flatten(),
        layers.Dense(128, activation= "relu"),
        layers.Dense(2, activation = "softmax")
      ])

  model.compile(loss="binary_crossentropy",
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=["accuracy"])
  return model

In [10]:
# FIRST MODEL FIT
epochs = 5

base_model = create_model()

#Fitting the model
history_1 = base_model.fit(
  train_data,
  validation_data=val_data,
  epochs=epochs,
  verbose=1
  )

base_model.save('models/base_model')

Epoch 1/5
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\danie\miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_13248\802985050.py", line 7, in <module>
    history_1 = model_1.fit(
  File "c:\Users\danie\miniconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\danie\miniconda3\lib\site-packages\tensorflow\python\eager\execute.py", line 54, in quick_execute
    tensors = pywrap_tfe.TFE_Py_Execute(ctx._handle, device_name, op_name,
tensorflow.python.framework.errors_impl.UnimplementedError: Graph execution error:

Detected at node 'sequential/conv2d/Relu' defined at (most recent call last):
    File "c:\Users\danie\miniconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\danie\minicond

In [ ]:
model_validation(base_model, val_data)

In [ ]:
#ploting Wrong Predictions
y_preds = base_model.predict(val_data)

id = 0
temp = 0
tempx = 0
for x,y in val_data:
  if id == 0:
    tempx = x
    temp = y
    id+=1
  else:
    tempx = np.concatenate([tempx,x])
    temp = np.concatenate([temp, y])

y_pred = (y_preds > 0.5)
temp = (temp > 0.5)

equal = (y_pred[:,0] == temp[:,0])

error_list = []
for i in range (0,equal.size):
  if not equal[i]:
    error_list.append((tempx[i], temp[i,0]))

plt.figure(figsize=(20,100))
i=0
for images, label in error_list:
      ax = plt.subplot(50, 10, i+1)
      plt.imshow(images)
      plt.title(class_names[label])
      plt.axis('off')
      i+=1